In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

from scipy.stats import norm
from scipy.stats import multivariate_normal

In [26]:
#load data into memory
data = pd.DataFrame(data = np.genfromtxt('sim3.dat', delimiter='  '), columns=['i','t','y_t','p_t'])

#set up lag
shift_data = data[['i','y_t','p_t']]
shift_data['t'] = data['t'] + 1
data = data.merge(shift_data,how='left',on=['i','t'],suffixes=['','-1'])
data = data.dropna()

In [79]:
#fix alpha ahead of time
alpha = np.random.normal(0,1 ,(1900,20))
    

class part_d(GenericLikelihoodModel):
    """class for evaluating question 3 part d"""
    
    def nloglikeobs(self, params, v=False):
        t0, t1, t2, sigma = params
        
        y = np.array([self.endog]).transpose()
        exog = self.exog
        
        #calculate the mean 'delta' for the inside good
        U1 = t0 + ([t1,t2]*exog).sum(axis=1)
        U1 = np.array([U1]).transpose()
        U1 = sigma*alpha + U1
        
        #calculate ll, for each simulation
        likelihood_sims = y*U1/(1+U1) + (1-y)/(1+U1)
        if v: print likelihood_sims.shape
        
        likelihood = likelihood_sims.prod(axis=0)
        if v: print likelihood.shape
            
        if v: raise Exception('Stop drop and roll')
        return - likelihood.sum()
        
        
        
        

    
    def fit(self, start_params=None, maxiter=100, maxfun=5000, **kwds):
        # we have one additional parameter and we need to add it for summary
        if start_params == None:
            start_params = [.5,.5,.5,.5]
        return super(part_d, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)
    


model_d = part_d(data['y_t'],data[['p_t','y_t-1']])

result_d = model_d.fit()
print(result_d.summary(xname=['theta_0', 'theta_1', 'theta_2', 'sigma']))

Optimization terminated successfully.
         Current function value: -0.000000
         Iterations: 9
         Function evaluations: 53
                                part_d Results                                
Dep. Variable:                    y_t   Log-Likelihood:                 0.0000
Model:                         part_d   AIC:                             4.000
Method:            Maximum Likelihood   BIC:                             15.10
Date:                Thu, 25 Oct 2018                                         
Time:                        09:56:51                                         
No. Observations:                1900                                         
Df Residuals:                    1898                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------

/home/erichschulman/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [61]:
s = np.random.normal(0,1 ,(1900,20))
t = np.random.normal(0,1 ,(1900,1))
print s.shape
print t.shape
print s + t

(1900, 20)
(1900, 1)
[[ 0.77993296 -0.55684051 -0.5804267  ... -0.21692896  0.88024991
   3.45831548]
 [ 0.81187536  1.30116026  2.4717956  ...  1.00261135  0.25495392
   0.93911685]
 [ 2.77424023  2.17276168  2.05435477 ...  2.14385061  1.14664104
   2.46350566]
 ...
 [ 1.47288051 -1.17222709  0.2247417  ... -0.10256321 -1.4774177
  -1.47303492]
 [ 1.25801197  3.00978794 -1.45741467 ...  0.94023556  3.11960666
   1.77865705]
 [ 1.2208654  -0.02527841  1.26710997 ...  2.40579482  2.8560812
   3.60237498]]
